## Streams
Java Streams API lets us manipulate collections of data in a declarative way. One effect is that the code size can be significantly reduced as the below example shows:

In [5]:
int[] numbers = {5, 7, 2, 9, 8, 1, 7};

// Copy the above array since we don't want to mutate it
int[] result = Arrays.copyOf(numbers, numbers.length);

// Sort it
Arrays.sort(result);

// Get sum of lowest 3
int sum = 0;
for(int i=0; i<3; i++){
    sum += result[i];
}

System.out.println(sum);

8


The same operation can be written in a concise manner using Streams

In [4]:
int[] numbers = {5, 7, 2, 9, 8, 1, 7};

int sum = Arrays.stream(numbers)
                .sorted()
                .limit(3)
                .sum();

System.out.println(sum);

8


Structure of a typical Stream computation:
**stream source** --> **intermediate operation(s)** -> **terminal operation**

### Stream Source

Object streams are the general kind of stream. To create an object stream:

In [ ]:
// From values
Stream<String> cityStream = Stream.of("Los Angeles", "Paris", "Tokyo", "Berlin");

// From arrays
String[] cities = new String[] { "Los Angeles", "Paris", "Tokyo", "Berlin" };
Stream<String> sameCityStream = Arrays.stream(cities);

// From collections
List<String> cityList = Arrays.asList(cities);
Stream<String> anotherCityStream = cityList.stream();

// Empty stream
Stream<Double> emptyStream = Stream.empty();

// From multiline String
String lines = "There are\nmultiple lines\nin this string";
Stream<String> linesStream = lines.lines();

// Infinite Stream using Supplier (has one method `T get()`)
Stream<UUID> infUUIDs = Stream.generate(UUID::randomUUID);

We have specialised Streams too: `IntStream`, `LongStream` and `DoubleStream` .  
![Stream Inheritance](images/stream_hierarchy.png)  

The stream method of Arrays has multiple overloaded versions as listed below (only a subset):
- `stream(int[] array)` returns `IntegerStream`
- `stream(double[] array)` returns `DoubleStream`
- `stream(T[] array)` returns `Stream<T>`

In [ ]:
int[] primes = new int[] { 2, 3, 5, 7, 11 };
IntStream primeStream = Arrays.stream(primes);

// list's stream method returns object streams
// because list itself is a collection of objects

To create the specialized streams we can use:

In [ ]:
DoubleStream doubleSteam = DoubleStream.of(3.56, 2.91, 8.314);

LongStream longStream = LongStream.range(1, 101);

We can also convert an object stream to specialised stream

In [ ]:
Stream<Integer> numbers = Stream.of(1, 4, 7, 8, 0, -5);
IntStream integerNumbers = numbers.mapToInt(i -> i);
// mapToLong and mapToDouble are the other two methods

// Range of integers
IntStream intStream = IntStream.range(1, 10001);  // exclusive

Or specialized stream to object stream:

In [ ]:
Stream<Double> doubles = DoubleStream.of(3.56, 2.91, 8.314).boxed();

There are methods available to create infinite streams:

In [ ]:
// Infinite Stream using iterator: initial value, next value function
IntStream infEvens = IntStream.iterate(0, i -> i + 2);

// Infinite random integers
IntStream infRandomInts = new Random().ints();

### Intermediate Operations
Intermediate operations on stream return stream. These operations can be chained together and are lazy evaluated (when a terminal operation is called).

| Operation                                                          | Description                                                                                                                                                                                  |
|--------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| sorted()                                                           | Returns a stream consisting of the elements of this stream, sorted according to natural order.                                                                                               |
| skip(long n)                                                       | Returns a stream consisting of the remaining elements of this stream after discarding the first n elements of the stream.                                                                    |
| peek(Consumer<? super T> action)                                   | Returns a stream consisting of the elements of this stream, additionally performing the provided action on each element as elements are consumed from the resulting stream.                  |
| limit(long n)                                                      | Returns a stream consisting of the distinct elements (according to `Object.equals(Object)`) of this stream.                                                                                  |
| distinct()                                                         | Working directory inside the container                                                                                                                                                       |
| filter(Predicate<? super T> predicate)                             | Returns a stream consisting of the elements of this stream that match the given predicate.                                                                                                   |
| map(Function<? super T, ? extends R> mapper)                       | Returns a stream consisting of the results of applying the given function to the elements of this stream.                                                                                    |
| flatMap(Function<? super T, ? extends Stream<? extends R>> mapper) | Returns a stream consisting of the results of replacing each element of this stream with the contents of a mapped stream produced by applying the provided mapping function to each element. |
| mapMulti(BiConsumer<? super T, ? super Consumer<R>> mapper)        | Returns a stream consisting of the results of replacing each element of this stream with multiple elements, specifically zero or more elements.                                              |

The lazy evalutaion characteristic can be seen in the below example:

In [ ]:
import java.util.stream.Stream;

// Nothing printed
Stream.of("d2", "a2", "b1", "b3", "c")
    .filter(s -> {
        System.out.println("filter: " + s);
        return true;
    });

Some operations can also be combined together while evaluating all the chained operations.

**`flatMap`** is an interesting intermediate operation. It transforms each element of a stream into zero or more elements and then flattens the resulting elements into a single stream. It accepts a `Function<? super T, ? extends Stream<? extends R>>` as argument (get and put principal: get element out of the original stream, put element into newer stream). Here `T` is element of original stream, `R` is the element of resulting flattened stream. In short, we need to supply a function that returns a stream from each element of the original stream. Flatmap is often used in case of nested data structures for example list of lists. We share a few examples below:

In [2]:
List<List<Integer>> nestedList = List.of(List.of(1), List.of(2, 3), List.of(4, 5, 6));
// Over every item of the outer list, we perform List.stream(), then we join the streams to one big flat stream
int sum = nestedList.stream().flatMap(Collection::stream).mapToInt(Integer::intValue).sum();
System.out.print(sum);

21

In [3]:
// This can be considered as list of list of characters
List<String> words = List.of("a collection of", "words that we", "want to count");
List<Integer> wordCount = words.stream().flatMap(element -> Arrays.stream(element.split(" ")).map(String::length)).toList();
System.out.print(wordCount);

[1, 10, 2, 5, 4, 2, 4, 2, 5]

**`mapMulti`** is an alternative to `flatMap`. With `mapMulti`, we push elements directly into a downstream consumer, avoiding creation of intermediate streams or collections, which can reduce memory and CPU overhead.

In [1]:
List<List<Integer>> anotherNestedList = List.of(List.of(1), List.of(2, 3), List.of(4, 5, 6));
// Notice usage of explicit type <Integer>
int sum = anotherNestedList.stream().<Integer>mapMulti(
    (item, consumer) -> item.forEach(consumer)).mapToInt(Integer::intValue).sum();
System.out.print(sum);

// Use mapMultiToInt to shorten
anotherNestedList.stream().mapMultiToInt((item, consumer) -> item.forEach(x -> x.intValue())).sum();

21

### Terminal Operations
These operations typically return a single value. Some examples:

In [9]:
// Matching terminal operations, return true or false
List<String> movies = List.of("One flew over the cuckoo's nest", "To kill a mockingbird", "Gone with the wind");

// Does any element in stream match the condition?
System.out.println(movies.stream().anyMatch(s -> s.startsWith("To")));

// Do all elements in stream match the condition?
System.out.println(movies.stream().allMatch(s -> s.startsWith("One")));

// Do none of the elements in stream match the condition?
System.out.println(movies.stream().noneMatch(s -> s.startsWith("Once")));

true
false
true


In [5]:
// Minimum, maximum, count
List<Integer> integers = List.of(12, 22, 45, 65, 5, 87);

// max() : Provide a Comparator, returns an Optional, since stream could have
// been empty
integers.stream().max((a, b) -> a - b).ifPresent(System.out::println);

// Counting
System.out.println(integers.stream().count());

87
6


`reduce` is one of the most important terminal operations. There are two overloaded variants:
- `reduce(BinaryOperator<T> accumulator)` returns `Optional<T>`
- `reduce(T identity, BinaryOperator<T> accumulator)` returns `T`

The first argument of the accumulator is the intermediate result, and the second argument is the stream element.

The below sequence of operations occur in case we use the version with identity element:
```
      1    5   ...
      |    |
IE - op - op - ... 
```

If we use the first version, three cases are possible:
- No element in the stream: return `Optional.empty()`
- One element: just return the element without applying the accumulator at all.
- Two or more elements: apply the accumulator to all of them and return the result.

The second variant will return atleast the identity element when the stream is empty.

In [9]:
import java.util.stream.*;

// Sum all the elements
integers.stream().reduce((a, b) -> a + b).ifPresent(System.out::println);

// Using below class for subsequent examples
class Person {
    String name;
    int age;

    Person(String name, int age) {
        this.name = name;
        this.age = age;
    }

    @Override
    public String toString() {
        return name;
    }
}

List<Person> persons = Arrays.asList(new Person("Max", 18), new Person("Peter", 23), new Person("Pamela", 23), new Person("David", 12));

// Person with maximum age
persons.stream().reduce((p1, p2) -> p1.age > p2.age ? p1 : p2).ifPresent(System.out::println);

// Get combined name using second form of reduce. Notice that this doesn't return Optional like the first form
System.out.println(persons.stream().reduce(new Person("", 0), (p1, p2) -> { p1.age += p2.age; p1.name += p2.name; return p1;}));

// Another demonstration of the second form of reduce
System.out.println(Stream.<String>empty().reduce("John Doe", (a,b) -> {a += b; return a;}));

236
Pamela
MaxPeterPamelaDavid
John Doe


`IntStream`, `LongStream`, `DoubleStream` provide some other terminal operations like `sum`, `average`, `summaryStatistics`.

The `collect` terminal operator is one of the most important terminal operator. With collect operation, we can use the result to form a collection, map, set, etc. We make use of the many static methods of the `Collectors` class.

In [ ]:
// Collect to a list
List<Integer> ageList = persons.stream().map(p -> p.age).sorted().collect(Collectors.toList()); // ArrayList

// Collect to an unmodifiable list
// nothing can be added after the list has been formed
List<Integer> immutableAgeList = persons.stream().map(a -> a.age).collect(Collectors.toUnmodifiableList());

// Collect to a Set
Set<String> nameSet = persons.stream().map(p -> p.name).collect(Collectors.toSet());

// There is also a unmodifiable set counterpart

In order to get a map, use the `toMap` collector.

In [ ]:
// Name -> Age map
Map<String, Integer> nameAgeMap = persons.stream().collect(Collectors.toMap(a -> a.name, a -> a.age));

// There is also a unmodifiable map counterpart

To join values, use the `joining` collector. It provides the nice benefit that joins are done between two elements.

In [ ]:
// Join together as string
String s = persons.stream().map(p -> p.name).collect(Collectors.joining(", ")); // No comma at the end!

// The same can be achieved using reduce, but is a bit clunky
String s_ = persons.stream().map(p -> p.name).reduce((a, b) -> a + ", " + b).get();

Using `partitioningBy` we can partition our stream into two segments. The two partitions are true and false partition and depend upon the condition we specify.

In [ ]:
Map<Boolean, List<Person>> agePartition = persons.stream().collect(Collectors.partitioningBy(p -> p.age > 30));

For multiple partitions, we can use `groupingBy`. By grouping, we are essentially creating buckets.

In [ ]:
// Age -> List of person
Map<Integer, List<Person>> ageToPersonMap = persons.stream()
    .collect(Collectors.groupingBy(p -> p.age));

// Person -> List of ages
// We employ an overloaded version of groupingBy which accepts another Collector
Map<String, List<Integer>> nameAgeGroup = persons.stream()
    .collect(Collectors.groupingBy(p -> p.name, Collectors.mapping(p -> p.age, Collectors.toList()))); 
// Why do we need toList collector in above case? Because the value is a list, if we want a set, we
// can use toSet collector

// Name -> name count
Map<String, Long> nameFrequency = persons.stream()
    .collect(Collectors.groupingBy(p -> p.name, Collectors.counting()));
// What if we want Integer and not Long?
Map<String, Integer> nameFrequency_ = persons.stream()
    .collect(Collectors.groupingBy(p -> p.name, Collectors.collectingAndThen(Collectors.counting(), Long::intValue)));

### Notes on Lambda Expressions
Lambda expressions passed into stream operations must adhere to the following:
- non-interfering: the stream source should not be modified during stream execution. Why? Because a stream may be executed in parallel. Multiple threads accessing a shared mutable state (the stream source) is not the best approach.
- stateless: lambdas passed to stream operations should be stateless (read or write any state that may change).  In the example below, if we use parallel stream, multiple threads would access the `twiceSeen` set without any coordination:

In [ ]:
HashSet<Integer> twiceSeen = new HashSet<>();
int[] result = new Random().ints().filter(e -> {
    twiceSeen.add(e * 2);
    return twiceSeen.contains(e);
}).limit(100).toArray();

- pure: lambda expressions should not have any side effect. One of the reason being sometimes the operation can be skipped, leading to side-effects also not being executed.

In [ ]:
int summation = IntStream.iterate(0, i -> i * 2).map(e -> {  // This map operation doesn't do much
    System.out.print(e);
    return e;
}).limit(10).sum();

## Parallel Streams
We can get a parallel stream by:

In [2]:
import java.util.stream.Stream;

// Converting existing sequential stream
Stream<Integer> stream = Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10);
stream.parallel().forEach(x -> System.out.print(x + " "));

7 4 5 1 2 10 9 3 8 6 

In [4]:
// Creating a parallel stream
Integer[] numbers = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 };
List<Integer> integerList = Arrays.asList(numbers);
Stream<Integer> stream = integerList.parallelStream();
stream.parallel().forEach(x -> System.out.print(x + " "));

3 1 2 9 4 8 6 7 5 10 

Using streams, the structure of concurrent code is same as the sequential one. As we can see from above example, there is just one switch to turn a sequential stream to a parallel one. What about the below code, will it create a parallel or a sequential stream?

In [5]:
Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    .parallel()
    .map(e -> (float)e )
    .sequential() // The last parallel or sequential operation decides
                  // since intermediate operations are lazily evaluated
    .forEach(x -> System.out.print(x + " "))   

1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 

Let's see which threads execute the intermediate operations

In [7]:
Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    .parallel()
    .map(e -> {
        System.out.println("Element=" + e + ", Thread=" + Thread.currentThread().getName());
        return e*e;
    })
    .forEach(x -> {})

Element=7, Thread=IJava-executor-3
Element=9, Thread=ForkJoinPool.commonPool-worker-3
Element=5, Thread=ForkJoinPool.commonPool-worker-5
Element=8, Thread=ForkJoinPool.commonPool-worker-27
Element=1, Thread=ForkJoinPool.commonPool-worker-7
Element=4, Thread=ForkJoinPool.commonPool-worker-21
Element=10, Thread=ForkJoinPool.commonPool-worker-13
Element=3, Thread=ForkJoinPool.commonPool-worker-9
Element=6, Thread=ForkJoinPool.commonPool-worker-31
Element=2, Thread=ForkJoinPool.commonPool-worker-17


In case of sequential streams:

In [8]:
// Main thread normally, Thread=IJava-executor-3 because this code
// is executed by IPython Java kernel
Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    .map(e -> {
        System.out.println("Element=" + e + ", Thread=" + Thread.currentThread().getName());
        return e*e;
    })
    .forEach(x -> {})

Element=1, Thread=IJava-executor-3
Element=2, Thread=IJava-executor-3
Element=3, Thread=IJava-executor-3
Element=4, Thread=IJava-executor-3
Element=5, Thread=IJava-executor-3
Element=6, Thread=IJava-executor-3
Element=7, Thread=IJava-executor-3
Element=8, Thread=IJava-executor-3
Element=9, Thread=IJava-executor-3
Element=10, Thread=IJava-executor-3


Parallel streams internally use `Common ForkJoinPool`. The number of threads is fixed (1 - number of cores). This can be modified using a vm argument.

From the `forEach` output we can see that in case of parallel stream, the order is result is different. Some methods are inherently unordered whereas some have ordered counterpart. Consider the code below:

In [9]:
Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    .parallel()
    .map(e -> e * 2)
    .forEachOrdered(x -> System.out.print(x + " "))   

2 4 6 8 10 12 14 16 18 20 

Even though we used parallel stream, the order of elements is maintained. This is still parallel execution. Let's see the threads involved:

In [11]:
Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    .parallel()
    .map(e -> {
        System.out.println("**Element=" + e + ", Thread=" + Thread.currentThread().getName());
        return e*e;
    })
    .forEachOrdered(e -> {
        System.out.println("--Element=" + e + ", Thread=" + Thread.currentThread().getName());
    })   

**Element=7, Thread=IJava-executor-5
**Element=10, Thread=ForkJoinPool.commonPool-worker-17
**Element=6, Thread=ForkJoinPool.commonPool-worker-9
**Element=8, Thread=ForkJoinPool.commonPool-worker-29
**Element=2, Thread=ForkJoinPool.commonPool-worker-15
**Element=5, Thread=ForkJoinPool.commonPool-worker-23
**Element=3, Thread=ForkJoinPool.commonPool-worker-13
**Element=1, Thread=ForkJoinPool.commonPool-worker-19
**Element=9, Thread=ForkJoinPool.commonPool-worker-25
**Element=4, Thread=ForkJoinPool.commonPool-worker-1
--Element=1, Thread=ForkJoinPool.commonPool-worker-19
--Element=4, Thread=ForkJoinPool.commonPool-worker-19
--Element=9, Thread=ForkJoinPool.commonPool-worker-19
--Element=16, Thread=ForkJoinPool.commonPool-worker-19
--Element=25, Thread=ForkJoinPool.commonPool-worker-19
--Element=36, Thread=ForkJoinPool.commonPool-worker-19
--Element=49, Thread=ForkJoinPool.commonPool-worker-19
--Element=64, Thread=ForkJoinPool.commonPool-worker-19
--Element=81, Thread=ForkJoinPool.commonP

We see that the two operations (map and forEachOrdered) are separately parallel. Compare this with:

In [12]:
Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    .parallel()
    .map(e -> {
        System.out.println("**Element=" + e + ", Thread=" + Thread.currentThread().getName());
        return e*e;
    })
    .forEach(e -> {
        System.out.println("--Element=" + e + ", Thread=" + Thread.currentThread().getName());
    })   

**Element=9, Thread=ForkJoinPool.commonPool-worker-25
**Element=1, Thread=ForkJoinPool.commonPool-worker-9
**Element=4, Thread=ForkJoinPool.commonPool-worker-3
--Element=16, Thread=ForkJoinPool.commonPool-worker-3
**Element=5, Thread=ForkJoinPool.commonPool-worker-17
**Element=10, Thread=ForkJoinPool.commonPool-worker-23
**Element=6, Thread=ForkJoinPool.commonPool-worker-13
**Element=8, Thread=ForkJoinPool.commonPool-worker-29
--Element=64, Thread=ForkJoinPool.commonPool-worker-29
**Element=2, Thread=ForkJoinPool.commonPool-worker-15
**Element=3, Thread=ForkJoinPool.commonPool-worker-1
--Element=9, Thread=ForkJoinPool.commonPool-worker-1
--Element=4, Thread=ForkJoinPool.commonPool-worker-15
--Element=36, Thread=ForkJoinPool.commonPool-worker-13
--Element=100, Thread=ForkJoinPool.commonPool-worker-23
--Element=25, Thread=ForkJoinPool.commonPool-worker-17
--Element=1, Thread=ForkJoinPool.commonPool-worker-9
--Element=81, Thread=ForkJoinPool.commonPool-worker-25
**Element=7, Thread=IJava-

The ordering is guaranteed by forEachOrdered because lists have ordering. If the stream was formed from a set, no ordering would have been present. Similar observation can be made when using the `findAny` method.

The reduce operation is also done parallely:

In [13]:
Stream.of(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    .parallel()
    .reduce(0, (total, x) -> {
        System.out.println("Total=" + total + ", x=" + x + ", Thread=" + Thread.currentThread().getName());
        return total + x;
    })

Total=0, x=3, Thread=ForkJoinPool.commonPool-worker-21
Total=0, x=4, Thread=ForkJoinPool.commonPool-worker-19
Total=0, x=1, Thread=ForkJoinPool.commonPool-worker-15
Total=0, x=5, Thread=ForkJoinPool.commonPool-worker-1
Total=0, x=9, Thread=ForkJoinPool.commonPool-worker-7
Total=0, x=10, Thread=ForkJoinPool.commonPool-worker-11
Total=0, x=2, Thread=ForkJoinPool.commonPool-worker-25
Total=0, x=8, Thread=ForkJoinPool.commonPool-worker-29
Total=0, x=7, Thread=IJava-executor-7
Total=1, x=2, Thread=ForkJoinPool.commonPool-worker-25
Total=9, x=10, Thread=ForkJoinPool.commonPool-worker-11
Total=4, x=5, Thread=ForkJoinPool.commonPool-worker-1
Total=3, x=9, Thread=ForkJoinPool.commonPool-worker-1
Total=0, x=6, Thread=ForkJoinPool.commonPool-worker-5
Total=6, x=7, Thread=ForkJoinPool.commonPool-worker-5
Total=3, x=12, Thread=ForkJoinPool.commonPool-worker-1
Total=8, x=19, Thread=ForkJoinPool.commonPool-worker-11
Total=13, x=27, Thread=ForkJoinPool.commonPool-worker-11
Total=15, x=40, Thread=ForkJ

55

Which is why if we pass in an incorrect identity element, the error is amplified in case of parallel stream.  

## Stream Internals


In the first phase data is sourced from a source and is represented as a `Spliterator`. For example, a stream using a `List` as source uses the `List` class' `Spliterator` implementation to iterate over the elements.

**Spliterator:** allows to traverse and partition a sequence. It contains two methods `tryAdvance` and `trySplit`. The second method is used for parallel streams.

In [ ]:
// tryAdvance steps through a sequence, returning false if no more elements are available
List<Integer> nums = IntStream.rangeClosed(1, 10).boxed().toList();
Spliterator<Integer> numsIterator = nums.spliterator();
while (numsIterator.tryAdvance(i -> System.out.println(i * i)));

`trySplit` attempts to divide the sequence into two equal halfs. This method may return null for any reason, including emptiness, inability to split after traversal has commenced, data structure constraints, and efficiency considerations.

In [ ]:
List<Integer> nums = IntStream.rangeClosed(1, 10).boxed().toList();
Spliterator<Integer> firstHalf = nums.spliterator();
Spliterator<Integer> secondHalf = firstHalf.trySplit();

while (firstHalf.tryAdvance(x -> System.out.println("First Half: " + x))) ;   // 6,7,8,9,10
while (secondHalf.tryAdvance(x -> System.out.println("Second Half: " + x))) ; // 1,2,3,4,5

Each operation on stream is stored in a `LinkedList` with every operation assigned flags:

| **Flag** 	| **Description**                                                                                                                                   |
|-----------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| SIZED    	| The size of the stream is known                                                                                                                   |
| DISTINCT 	| The elements in the stream are unique, no duplicates.                                                                                             |
| SORTED   	| Elements are sorted in a natural order.                                                                                                           |
| ORDERED  	| Whether the stream has a meaningful encounter order; meaning that the order in which the elements are streamed should be preserved on collection. |

Each operation will clear, set or preserve different flags; this is quite important because this means that each stage knows what effectsis caused by itself on these flags and this will be used to make the optimisations. For example,`map` will clear `SORTED` and `DISTINCT` bits because data may have changed; however it will always preserve `SIZED` flag. `filter` will clear `SIZED` flag because size of the stream may have changed, but it’ll always preserve `SORTED` and `DISTINCT` flags because filter will never modify the structure of the data.

When terminal operation gets executed, `Stream` selects an execution plan. There are two possible scenarios i) all stages are stateless ii) no all stages are stateless.  

**Stateless operations:** doesn’t need to know about any other element to be able to emit a result. Examples of stateless operations are: `filter`, `map` or `flatMap`.  
**Stateful operations** need to know about all the elements before emitting a result. Examples of stateful operations are: `sorted`, `limit` or `distinct`.

If all operations are stateless then the Stream can be processed in one go. On the other hand, if it contains stateful operations, the pipeline is divided into sections using the stateful operations as delimiters.